<a href="https://colab.research.google.com/github/joaop-n12/Machine_Learning/blob/main/Regress%C3%A3o_Polinomial_e_teste_de_Overfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
import scipy.stats as stats

df = pd.read_csv('/content/insurance - insurance.csv (1).csv')
df

,age,sex,bmi,children,smoker,region,charges
0,35.0,male,30.50,1,no,southwest,"10414,89"
1,18.0,male,34.10,0,no,southeast,"11370,11"
2,19.0,male,19.80,0,no,southwest,"12415,65"
3,19.0,male,20.70,0,no,southwest,"12428,16"
4,19.0,male,27.60,0,no,southwest,"12524,07"
...,...,...,...,...,...,...,...
490,34.0,female,23.56,0,no,northeast,"499237,64"
491,34.0,female,26.73,1,no,southeast,"500278,27"
492,37.0,male,29.64,0,no,northwest,"502814,66"
493,35.0,male,17.86,1,no,northwest,"511650,04"


In [ ]:
df.isnull().sum()

,0
age,54
sex,0
bmi,106
children,0
smoker,0
region,0
charges,0


In [ ]:
df['region'].value_counts()

,count
region,
southwest,263
southeast,130
northeast,52
northwest,50


In [ ]:
df1 = df.copy()
df1['sex'] = df.apply( lambda row: 1 if row['sex']=='male' else 0 , axis=1)
df1['smoker'] = df.apply( lambda row: 1 if row['smoker']=='yes' else 0 , axis=1)
df1['southwest'] = df.apply( lambda row: 1 if row['region']=='southwest' else 0 , axis=1)
df1['southeast'] = df.apply( lambda row: 1 if row['region']=='southeast' else 0 , axis=1)
df1['northeast'] = df.apply( lambda row: 1 if row['region']=='northeast' else 0 , axis=1)
df1['northwest'] = df.apply( lambda row: 1 if row['region']=='northwest' else 0 , axis=1)
df1 = df1.drop('region', axis=1)
df1['charges'] = df1['charges'].str.replace(',', '').astype(float)
filtered_df = df1[df1['age'].notnull() & df1['bmi'].notnull()]
filtered_df

,age,sex,bmi,children,smoker,charges,southwest,southeast,northeast,northwest
0,35.0,1,30.50,1,0,1041489.0,1,0,0,0
1,18.0,1,34.10,0,0,1137011.0,0,1,0,0
2,19.0,1,19.80,0,0,1241565.0,1,0,0,0
3,19.0,1,20.70,0,0,1242816.0,1,0,0,0
4,19.0,1,27.60,0,0,1252407.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
490,34.0,0,23.56,0,0,49923764.0,0,0,1,0
491,34.0,0,26.73,1,0,50027827.0,0,1,0,0
492,37.0,1,29.64,0,0,50281466.0,0,0,0,1
493,35.0,1,17.86,1,0,51165004.0,0,0,0,1


In [ ]:
x = filtered_df[['age','sex','bmi','children','smoker','southwest','southeast','northeast','northwest']].to_numpy()
y = filtered_df['charges'].to_numpy()
model = LinearRegression()
model.fit(x, y)
score = model.score(x, y)
print(score)

0.4014041130407039


In [ ]:
filtered_df = df1[(df1['age'].notnull()) & (df1['bmi'].notnull())]
x = filtered_df[['sex','bmi','children','smoker','southwest','southeast','northeast','northwest']].to_numpy()
y = filtered_df['age'].to_numpy()
model_age = LinearRegression()
model_age.fit(x, y)
model_age.score(x, y)


0.3286528588115061

In [ ]:
def completar_age(row):
  if not pd.isna(row['bmi']) and pd.isna(row['age']):
    campos = row[['sex','bmi','children','smoker','southwest','southeast','northeast','northwest']]
    return round(model_age.predict([campos])[0])
  return row['age']
df1['novo_age'] = df1.apply(completar_age, axis=1)
df1

,age,sex,bmi,children,smoker,charges,southwest,southeast,northeast,northwest,novo_age
0,35.0,1,30.50,1,0,1041489.0,1,0,0,0,35.0
1,18.0,1,34.10,0,0,1137011.0,0,1,0,0,18.0
2,19.0,1,19.80,0,0,1241565.0,1,0,0,0,19.0
3,19.0,1,20.70,0,0,1242816.0,1,0,0,0,19.0
4,19.0,1,27.60,0,0,1252407.0,1,0,0,0,19.0
...,...,...,...,...,...,...,...,...,...,...,...
490,34.0,0,23.56,0,0,49923764.0,0,0,1,0,34.0
491,34.0,0,26.73,1,0,50027827.0,0,1,0,0,34.0
492,37.0,1,29.64,0,0,50281466.0,0,0,0,1,37.0
493,35.0,1,17.86,1,0,51165004.0,0,0,0,1,35.0


In [ ]:
# filtered_df = df1[(df1['age'].notnull()) & (df1['bmi'].notnull())]
x = filtered_df[['sex','age','children','smoker','southwest','southeast','northeast','northwest']].to_numpy()
y = filtered_df['bmi'].to_numpy()
model_bmi = LinearRegression()
model_bmi.fit(x, y)
model_bmi.score(x, y)


0.12119653074632508

In [ ]:
def completar_bmi(row):
  if not pd.isna(row['age']) and pd.isna(row['bmi']):
    campos = row[['sex','age','children','smoker','southwest','southeast','northeast','northwest']]
    return round(model_bmi.predict([campos])[0])
  return row['bmi']
df1['novo_bmi'] = df1.apply(completar_bmi, axis=1)
df1

,age,sex,bmi,children,smoker,charges,southwest,southeast,northeast,northwest,novo_age,novo_bmi
0,35.0,1,30.50,1,0,1041489.0,1,0,0,0,35.0,30.50
1,18.0,1,34.10,0,0,1137011.0,0,1,0,0,18.0,34.10
2,19.0,1,19.80,0,0,1241565.0,1,0,0,0,19.0,19.80
3,19.0,1,20.70,0,0,1242816.0,1,0,0,0,19.0,20.70
4,19.0,1,27.60,0,0,1252407.0,1,0,0,0,19.0,27.60
...,...,...,...,...,...,...,...,...,...,...,...,...
490,34.0,0,23.56,0,0,49923764.0,0,0,1,0,34.0,23.56
491,34.0,0,26.73,1,0,50027827.0,0,1,0,0,34.0,26.73
492,37.0,1,29.64,0,0,50281466.0,0,0,0,1,37.0,29.64
493,35.0,1,17.86,1,0,51165004.0,0,0,0,1,35.0,17.86


In [ ]:
df1 = df1.drop(['age','bmi'], axis=1)
isnull = df1.isnull().sum()
print(isnull)

sex          0
children     0
smoker       0
charges      0
southwest    0
southeast    0
northeast    0
northwest    0
novo_age     0
novo_bmi     0
dtype: int64


In [ ]:
filtered_df = df1[(df1['novo_age'].notnull()) & (df1['novo_bmi'].notnull())]
x = filtered_df[['novo_age','sex','novo_bmi','children','smoker','southwest','southeast','northeast','northwest']].to_numpy()
y = filtered_df['charges'].to_numpy()
model = LinearRegression()
model.fit(x, y)
model.score(x, y)
score = model.score(x, y)
print(score)

0.5127630400505596


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state=42)

model = LinearRegression()
model.fit(x_train, y_train)
train_score = model.score(x_train, y_train)
test_score = model.score(x_test, y_test)

print("R² treino Linear: ", train_score)
print("R² teste Linear: ", test_score)

threshold = 0.2
calculated_threshold = 0
degree = 0
while calculated_threshold < threshold:
  # Criando variável polinomial para o X
  degree += 1
  poly_features = PolynomialFeatures(degree=degree)
  x_train_poly = poly_features.fit_transform(x_train)
  polynomial_model = LinearRegression()
  polynomial_model.fit(x_train_poly, y_train)
  polynomial_train_score = polynomial_model.score(x_train_poly, y_train)

  x_test_poly = poly_features.fit_transform(x_test)
  polynomial_test_score = polynomial_model.score(x_test_poly, y_test)
  calculated_threshold = polynomial_train_score / polynomial_test_score - 1

  print("")
  print("Grau", degree)
  print("R² treino Polynomial: ", polynomial_train_score)
  print("R² teste Polynomial: ", polynomial_test_score)
  print("calculated_treshold: ", calculated_threshold)

  if polynomial_test_score < 0:
    print("Estouro do R² teste Polynomial")
    break


R² treino Linear:  0.4963864086897679
R² teste Linear:  0.4960090203795391

Grau 1
R² treino Polynomial:  0.4963864086897679
R² teste Polynomial:  0.49600902037953887
calculated_treshold:  0.0007608496916855501

Grau 2
R² treino Polynomial:  0.6961855122634528
R² teste Polynomial:  0.5191728505074227
calculated_treshold:  0.34095130664676265
